# Sanity check del notebook para IQLAgent
Este notebook hace un sanity check de la implementación de `IQLAgent` en todos los juegos disponibles durante al menos 10 episodes.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from games.rps import RPS
from games.mp import MP
from games.blotto import Blotto
from games.foraging import Foraging
from agents.iql_agent import IQLAgent, IQLAgentConfig

## Definir juegos y configuraciones
Define los juegos a probar y sus configuraciones.

In [2]:
games_to_test = [
    {"name": "RPS", "game": RPS, "config": {}},
    {"name": "MP", "game": MP, "config": {}},
    {"name": "Blotto", "game": Blotto, "config": {"S": 3, "N": 2}},
    {"name": "Foraging", "game": Foraging, "config": {"config": "Foraging-5x5-2p-1f-v3", "seed": 1}},
]

def get_game_instance(game_entry):
    if game_entry["name"] == "Blotto":
        return game_entry["game"](**game_entry["config"])
    elif game_entry["name"] == "Foraging":
        return game_entry["game"](**game_entry["config"])
    else:
        return game_entry["game"]()

## Inicializar IQLAgent para cada juego
Para cada juego, inicializa el `IQLAgent` para todos los agentes en el entorno.

In [ ]:
def create_agents(game):
    config = IQLAgentConfig(alpha=0.1, min_epsilon=0.01, seed=1)
    return {agent: IQLAgent(game=game, agent=agent, config=config) for agent in game.agents}

## Correr episodios para cada juego
Ejecuta al menos 10 episodes por juego y recolecta las cumulative rewards.

In [ ]:
def play_episodes(game, agents, episodes=10):
    recompensas = {agent: 0.0 for agent in game.agents}
    for ep in range(episodes):
        game.reset()
        for agent in agents.values():
            agent.reset()
        terminado = False
        while not getattr(game, 'done', lambda: True)() and not all(game.terminations.values()):
            acciones = {agent: agents[agent].action() for agent in game.agents}
            game.step(acciones)
            for agent in game.agents:
                agents[agent].update()
                recompensas[agent] += game.reward(agent)
    return recompensas

## Mostrar resultados
Mostrar las cumulative rewards para cada Agent en cada game después de 10 episodes.

In [5]:
resultados = {}
for entry in games_to_test:
    print(f"\nProbando {entry['name']}...")
    game = get_game_instance(entry)
    agents = create_agents(game)
    recompensas = play_episodes(game, agents, episodes=10)
    resultados[entry['name']] = recompensas
    for agent, recompensa in recompensas.items():
        print(f"Agente {agent}: Recompensa total en 10 episodios: {recompensa}")


Probando RPS...
Agente agent_0: Recompensa total en 10 episodios: 0.0
Agente agent_1: Recompensa total en 10 episodios: 0.0

Probando MP...
Agente agent_0: Recompensa total en 10 episodios: 0.0
Agente agent_1: Recompensa total en 10 episodios: 0.0

Probando Blotto...
Agente agent_0: Recompensa total en 10 episodios: 0.0
Agente agent_1: Recompensa total en 10 episodios: 0.0

Probando Foraging...
Agente agent_0: Recompensa total en 10 episodios: 3.0
Agente agent_1: Recompensa total en 10 episodios: 0.0


/opt/homebrew/Caskroom/miniconda/base/envs/py311/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:245: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'list'>
  logger.warn(
